In [1]:
import collections
import d2lzh as d2l
import math 
from mxnet import autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,nn
import random
import sys
import time
import zipfile

<font size=4 color='blue'>**1、预处理数据集**</font>

In [2]:
with zipfile.ZipFile("../d2l-zh/data/ptb.zip",'r') as zin:
    zin.extractall("../data/")
    
with open("../data/ptb/ptb.train.txt",'r') as f:
    lines = f.readlines()
    
    raw_dataset = [st.split() for st in lines]
    
'# sentences: %d'%len(raw_dataset)

'# sentences: 42068'

In [3]:
# 对数据集的前3个句子，打印每个句子的词数和前5个词
for st in raw_dataset[:3]:
    print("# tokens:",len(st),st[:5])

# tokens: 24 ['aer', 'banknote', 'berlitz', 'calloway', 'centrust']
# tokens: 15 ['pierre', '<unk>', 'N', 'years', 'old']
# tokens: 11 ['mr.', '<unk>', 'is', 'chairman', 'of']


<font size=4 color='blue'>**1.1、建立词索引**</font>

In [4]:
# 只保留出来5次以上的词
counter = collections.Counter([tk for st in raw_dataset for tk in st])
counter = dict(filter(lambda x:x[1]>=5, counter.items()))

# 将词映射到整数索引
idx_to_token = [tk for tk,_ in counter.items()] ## 词
token_to_idx = {tk:idx for idx,tk in enumerate(idx_to_token)} #{词1:索引1, 词2:索引2}

# 将数据集转换为索引
dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset]

# 字符数
num_tokens = sum([len(st) for st in dataset])

'#tokens: %d' % num_tokens

'#tokens: 887100'

<font size=4 color='blue'>**1.2、二次采样**</font>

文本数据中一般会出现一些高频词，如英文中的“the” “a” 和“in” 。通常来说，在一个背景窗口中，一个词（如“chip”）和较低频词（如“microprocessor”）同时出现比和较高频词（如“the”）同时出现对训练词嵌入模型更有益。因此，训练词嵌入模型时可以对词进行二次采样（subsampling）。具体来说，数据集中每个被索引词 $w_i$ 将有一定概率被丢弃，该丢弃概率为
#### $$P(w_i) = max\Big(1 - \sqrt{\frac{t}{f(w_i)}}, 0\Big)$$
其中，$f(w_i)$ 是数据集中词 $w_i$ 的个数与总词数之比，常数 $t$ 是一个超参数（实验室中设为 $10^{-4}$）。可见，**只有当 $f(w_i)>t$ 时，我们才有可能在二次采样中丢弃词 $w_i$，并且越高频的词被丢弃的概率越大。** 

In [5]:
def discard(idx):
    return random.uniform(0,1) < 1 - math.sqrt(1e-4 / counter[idx_to_token[idx]] * num_tokens)

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset]

'# tokens: %d' % sum([len(st) for st in subsampled_dataset])

'# tokens: 375795'

In [6]:
# 比较一个词在二次采样前后出现在数据集中的次数。可见高频词 “the” 的采样率不足 1/20

def compare_counts(token):
    return "# %s: before = %d, after = %d" %(token, sum([st.count(token_to_idx[token]) for st in dataset])
                                                   ,sum([st.count(token_to_idx[token] for st in subsampled_dataset)]))

compare_counts("the")

'# the: before = 50770, after = 0'

<font size=4 color='blue'>**1.3、提取中心词和背景词**</font>

将与中心词距离不超过背景窗口大小的词作为它的背景词。


In [7]:
# 提取所有中心词和背景词。它每次在整数 1 和 max_window_size (最大背景窗口) 之间随机均匀采样一个整数作为背景窗口大小

def get_centers_and_contexts(dataset, max_window_size):
    centers, contexts = [],[]
    for st in dataset:
        if len(st) < 2: # 每个句子至少要有2个词才可能组成一对 “中心词 - 背景词”
            continue 
        centers += st
        
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, center_i - window_size),  ## 如果当前索引 center_i 小于 window_size ,就从头（即0 开始）
                                 min(len(st), center_i + 1 + window_size)))
            indices.remove(center_i) # 将中心词排除在背景词之外
            contexts.append([st[idx] for idx in indices])
    return centers, contexts

# 设置最大背景窗口为5
all_centers, all_contexts = get_centers_and_contexts(subsampled_dataset, 5)

In [8]:
# test
# 创建一个人工数据集，其中含有词数分别为7和3的两个句子。 设最大背景窗口为2，打印所有中心词和它们的背景词
tiny_dataset = [list(range(7)), list(range(7,10))]
print("dataset:", tiny_dataset)

for center,context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print("center", center, "has contexts" ,context)

dataset: [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1]
center 1 has contexts [0, 2, 3]
center 2 has contexts [1, 3]
center 3 has contexts [1, 2, 4, 5]
center 4 has contexts [2, 3, 5, 6]
center 5 has contexts [4, 6]
center 6 has contexts [4, 5]
center 7 has contexts [8]
center 8 has contexts [7, 9]
center 9 has contexts [8]


<font size=4 color='blue'>**2、负采样**</font>

使用负采样进行近似训练。 对于一对中心词和背景词，我们随机采样  $K$ 个噪声词（实验中设 $K=5$）。根据 word2vec 论文的建议，噪声词采样概率 $P(w)$ 设为 $w$ 词频 与总词频之比的0.75次方。

设背景词 $w_o$ 出现在中心词 $w_c$ 的一个背景窗口为事件 $P$，我们根据分布 $P(w)$ 采样 $K$ 个未出来在该背景窗口中的词，即噪声词。


In [9]:
def get_negatives(all_contexts, sampling_weights, K):
    ''' all_contexts : 从get_centers_and_contexts 提取出的所有背景词
        sampling_weights: 每个词的词频的0.75次方
    '''
    all_negatives, neg_candidates, i = [],[],0
    
    population = list(range(len(sampling_weights)))
    
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # 根据每个词的权重（sampling_weights）随机生成 k 个词的索引作为噪声词
                # 为了高效率计算，可以将 k 设得稍大一些
                i,neg_candidates = 0, random.choices(population, sampling_weights, k = int(1e5))
                
            neg, i = neg_candidates[i], i+1
            # 噪声词不是背景词
            if neg not in set(contexts):
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

sampling_weights = [counter[w]**0.75 for w in idx_to_token] 
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

<font size=4 color='blue'>**3、读取数据集**</font>

从数据集中提取所有中心词 all_centers, 以及每个中心词对应的背景词 all_contexts 和噪声词 all_negatives 。我们通过随机小批量来读取它们。

在一个小批量数据中，第 $i$ 个样本包括一个中心词以及它所对应的 $n_i$ 个背景词和 $m_i$ 个噪声词。由于每个样本的背景窗口大小可能不一样，其中背景词与噪声词个数之和 $n_i + m_i$ 也会不同。要构造小批量时，我们将每个样本的背景词和噪声词连结在一起，并添加填充项0直到连结后的长度相同，即长度均为 $max n_i + m_i$ （max_len 变量）。为了避免填充项对损失函数计算的影响，我们构造了掩码变量 masks, 其每一个元素分别与连结后的背景词和噪声词 contexts_negatives 中的元素一一对应。 当contexts_negatives 变量中的某个元素为填充项时， 相同位置的掩码变量 masks 中的元素取0， 否则 取1。为了区分与正类与负类，我们还需要将contexts_negatives 变量中的背景词和噪声词区分开来。依据掩码变量的构造思路，我们只需要创建与 contexts_negatives 变量形状相同的标签变量 labels, 并将与背景词（正类）对应的元素设为1，其余清0.

In [10]:
# 小批量读取函数 batchify  它的小批量输入 data 是一个长度为批量大小的列表，其中每个元素分别包含中心词 center、背景词 context 和 噪声词 negative 。
# 该函数返回的小批量数据符合我们需要的格式，包含了掩码变量。

def batchify(data):
    max_len = max(len(c) + len(n) for _,c,n in data)
    centers,contexts_negatives,masks,labels = [],[],[],[]
    for center,context,negative in data:
        cur_len = len(context) + len(negative)
        centers += [center]
        contexts_negatives += [context + negative + [0]*(max_len - cur_len)]
        masks += [[1] * cur_len + [0]*(max_len - cur_len)]
        labels += [[1] * len(context) + [0] *(max_len - len(context))]
    return (nd.array(centers).reshape((-1,1)), nd.array(contexts_negatives) , nd.array(masks), nd.array(labels))
        

In [11]:
# 打印读取的第一批量中各个变量的形状
batch_size = 512
num_workers = 0 if sys.platform.startswith("win32") else 4
dataset = gdata.ArrayDataset(all_centers, all_contexts, all_negatives)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle = True, batchify_fn = batchify, num_workers = num_workers)

for batch in data_iter:
    for name,data in zip(['centers','contexts_negatives','masks','labels'], batch):
        print(name,"shape:",data.shape)
    break

centers shape: (512, 1)
contexts_negatives shape: (512, 60)
masks shape: (512, 60)
labels shape: (512, 60)


#### <font size=4 color='blue'>**4、跳字模型**</font>

#### <font size=4 color='blue'>**4.1、嵌入层**</font>

获取词嵌入的层称为嵌入层。嵌入层的权重是一个矩阵，其行数为词典大小（input_dim)，列数为每个词向量的维度（output_dim）。

嵌入词的输入为词的索引。输入一个词的索引i，嵌入层返回权重矩阵的第 i 行作为它的词向量。

In [12]:
embed = nn.Embedding(input_dim = 20, output_dim = 4)
embed.initialize()
embed.weight

Parameter embedding0_weight (shape=(20, 4), dtype=float32)

In [13]:
x = nd.array([[1,2,3],[1,5,6]])
embed(x)


[[[ 0.01438687  0.05011239  0.00628365  0.04861524]
  [-0.01068833  0.01729892  0.02042518 -0.01618656]
  [-0.00873779 -0.02834515  0.05484822 -0.06206018]]

 [[ 0.01438687  0.05011239  0.00628365  0.04861524]
  [ 0.0408415   0.04370362  0.00404529 -0.0028032 ]
  [ 0.00952624 -0.01501013  0.05958354  0.04705103]]]
<NDArray 2x3x4 @cpu(0)>

#### <font size=4 color='blue'>**4.2、小批量乘法**</font>

使用小批量乘法运算 batch_dot 对两个小批量中的矩阵一一做乘法。假设第一个小批量包含 n 个形状为 $a\times b$的矩阵 $X_i,\cdots,X_n$，第二个小批量包含 $n$ 个形状为 $b\times c$ 的矩阵 $Y_1,\cdots,Y_n$。这两个小批量的矩阵乘法输出为 $n$ 个形状为 $a\cdots c$ 的矩阵 $X_1Y_1,\cdots,X_nY_n$。因此，给定两个形状分别为（n,a,b）和(n,b,c)的NDARrray，小批量乘法输出的形状为(n,a,c)。 

In [14]:
X = nd.ones((2,1,4))
Y = nd.ones((2,4,6))
nd.batch_dot(X,Y).shape

(2, 1, 6)

#### <font size=4 color='blue'>**4.3、跳字模型的前向计算**</font>

** 跳字模型假设基于某个词来生成它在文本序列周围的词。

在前向计算中，跳字模型的输入包含中心词索引center以及连结的背景词与噪声词索引contexts_and_negatives。其中，center变量的形状为(批大小，1),而contexts_and_negatives变量的形状大小为(批大小, max_len)。这两个变量先通过词嵌入层分别由词索引变换为词向量，再通过小批量乘法得到形状为(批大小, 1, max_len) 的输出。输出中的每个元素是中心词与背景词向量或噪声词向量的内积。

In [15]:
def skip_gram(center,contexts_and_negatives, embed_v,embed_u):
    v = embed_v(center) # 中心词
    u = embed_u(contexts_and_negatives) # 背景词
    
    pred = nd.batch_dot(v,u.swapaxes(1,2))
    
    return pred

#### <font size=4 color='blue'>**5、训练模型**</font>

#### <font size=4 color='blue'>**5.1、二元交叉熵损失函数**</font>


In [16]:
loss = gloss.SigmoidBinaryCrossEntropyLoss()

<font size=3 color='green'>**可以通过掩码变量指定小批量中参与损失函数计算的部分预测值和标签：当掩码为1时，相应位置的预测值和标签将参与损失函数的计算；当掩码值为0时，相应位置的预测值和标签则不会参与损失函数的计算。掩码变量可用于避免填充项对损失函数的计算的影响。**</font>

In [17]:
pred = nd.array([[1.5,0.3,-1,2],[1.1,-0.6,2.2,0.4]])

# 标签变量label 中的1和0分别代表背景词和噪声词
label = nd.array([[1,0,0,0],[1,1,0,0]])
mask = nd.array([[1,1,1,1],[1,1,1,0]]) # 掩码变量
loss(pred, label, mask) * mask.shape[1] / mask.sum(axis = 1)



[0.8739896 1.2099689]
<NDArray 2 @cpu(0)>

In [18]:
# 实现二元交叉熵损失函数的计算，并根据掩码变量 mask计算掩码为1的预测值和标签的损失

def sigmoid(x):
    return -math.log(1/(1 + math.exp(-x)))

print("%.7f" %((sigmoid(1.5) + sigmoid(-0.3) + sigmoid(1) + sigmoid(-2)) / 4))
print("%.7f" %((sigmoid(1.1) + sigmoid(-0.6) + sigmoid(-2.2)) / 3))

0.8739896
1.2099689


#### <font size=4 color='blue'>**5.2、初始化模型参数**</font>

分别构造中心词和背景词的嵌入层，并将超参数词向量维度 embed_size 设置成100

In [19]:
embed_size = 100
net = nn.Sequential()
net.add(nn.Embedding(input_dim = len(idx_to_token), output_dim = embed_size),
        nn.Embedding(input_dim = len(idx_to_token), output_dim = embed_size))

#### <font size=4 color='blue'>**5.3、定义训练函数**</font>


In [22]:
def train(net,lr,num_epochs):
    ctx = d2l.try_gpu()
    net.initialize(ctx = ctx, force_reinit = True)
    trainer = gluon.Trainer(net.collect_params(),"adam", {"learning_rate": lr})
    
    for epoch in range(num_epochs):
        start, l_sum, n = time.time(),0.0,0
        for batch in data_iter:
            center,context_negative,mask,label = [data.as_in_context(ctx) for data in batch]
            with autograd.record():
                pred = skip_gram(center, context_negative, net[0], net[1])
                # 使用掩码变量 mask 来避免填充项对损失函数计算的影响
                l = (loss(pred.reshape(label.shape), label, mask) * mask.shape[1] / mask.sum(axis = 1))
            l.backward()
            trainer.step(batch_size)
            l_sum += l.sum().asscalar()
            n += l.size
        print("epoch %d, loss %.2f, time %.2fs" %(epoch +1, l_sum/n, time.time() - start))
            
            

In [23]:
train(net, 0.005, 5)

epoch 1, loss 0.46, time 25.09s
epoch 2, loss 0.39, time 21.33s
epoch 3, loss 0.35, time 20.19s
epoch 4, loss 0.32, time 19.80s
epoch 5, loss 0.31, time 19.78s


#### <font size=4 color='blue'>**6、应用词嵌入模型**</font>

训练好词嵌入模型之后，可以根据两个词向量的余弦相似度表示词与词之间在语义上的相似度。

In [27]:
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data()
    x = W[token_to_idx[query_token]]
    
    # 添加e-9 是为了数值稳定性
    cos = nd.dot(W,x) / (nd.sum(W*W, axis = 1) * nd.sum(x*x) + 1e-9).sqrt()
    topk = nd.topk(cos, k = k+1, ret_typ = "indices").asnumpy().astype("int32")
    for i in topk[1:]:
        print("cosine sim = %.3f: %s" % (cos[i].asscalar(), (idx_to_token[i])))
        
get_similar_tokens("chip", 3, net[0])

cosine sim = 0.538: dell
cosine sim = 0.528: microprocessors
cosine sim = 0.520: intel
